In [6]:
import os
import sys
import shutil
import argparse
import numpy as np
import tensorflow as tf
import scipy.io as sio
import glob
# sys.path.append('git/_framework')
from utils import util as U
from core.data_provider import DataProvider
from core.trainer_tf_uni import Trainer
from core.data_processor import SimpleImageProcessor
from core.learning_rate import StepDecayLearningRate
from model.model_wgan_uni import GANModel
from model.gan_10k import Generator, Discriminator
import timeit

In [7]:
start = timeit.default_timer()
epochs = 1
batch_size = 1
mini_batch_size = 1
eval_batch_size = 1

eval_frequency = 2
use_bn = False
use_res = True
learning_rate = 0.001
g_alpha = 40
g_lambda = 10
g_beta = 10
dropout = 0
resize = None
output_path = '/home/padamnoo/Desktop/Results/wgan_10k_640_zm_mm'
output_path = '{}_batch{}_bn{}_dp{}_a{}_la{}_lr{}'.format(output_path, batch_size, int(use_bn), dropout, g_alpha, g_lambda, learning_rate)
saved_filelists = 'std_sex_640_160_62.mat'

In [8]:
import platform

if platform.system() == 'Windows':
    data_path = './'
    output_path = 'results/' + output_path
if platform.system() == 'Linux':
    data_path = './'
    output_path = 'results/' + output_path

In [9]:
org_path = data_path + '/std96'
saved_filelists = data_path + '/' + saved_filelists
# load filenames
# file_mat = sio.loadmat(saved_filelists)

# load filenames
data_path = '/home/padamnoo/Desktop/Data/ModelData/'
train_list_T1 = glob.glob(data_path + 'train/T1/*')
valid_list_T1 = glob.glob(data_path + 'valid/T1/*')
train_list_T2 =glob.glob(data_path + 'train/T2/*')
valid_list_T2 = glob.glob(data_path + 'valid/T2/*')
# test_list = glob.glob(data_path + '/test/*_img.png')

# set key for input images and labels
org_suffix = '_img.png'
lab_suffix = '_lab.png'

In [10]:
len(train_list_T2)

1294

In [11]:
pre = {org_suffix: [ 'zero-mean', ('channelcheck', 1)],
       lab_suffix: [('one-hot', [0, 63, 127, 191, 255]), ('channelcheck', 5)]
       }

pre_ = {org_suffix: [ 'zero-mean', ('channelcheck', 1)],
        lab_suffix: [('one-hot', [0, 63, 127, 191, 255]), ('channelcheck', 5)]
       }

In [12]:
processor = SimpleImageProcessor(pre=pre)
processor_no_p = SimpleImageProcessor(pre=pre_)

In [13]:
train_provider_T1_t = DataProvider(train_list_T1, [org_suffix, lab_suffix],
                        is_pre_load=False,
                        is_shuffle=True,
                        # temp_dir=output_path,
                        processor=processor)

valid_provider_T1_t = DataProvider(valid_list_T1, [org_suffix, lab_suffix],
                        is_pre_load=False,
                        # temp_dir=output_path,
                        processor=processor)

train_provider_T1 = DataProvider(train_list_T1, [org_suffix],
                        is_pre_load=False,
                        is_shuffle=True,
                        # temp_dir=output_path,
                        processor=processor)

valid_provider_T1 = DataProvider(valid_list_T1, [org_suffix],
                        is_pre_load=False,
                        # temp_dir=output_path,
                        processor=processor)

train_provider_T2 = DataProvider(train_list_T2, [org_suffix],
                        is_pre_load=False,
                        is_shuffle=True,
                        # temp_dir=output_path,
                        processor=processor)

valid_provider_T2 = DataProvider(valid_list_T2, [org_suffix],
                        is_pre_load=False,
                        # temp_dir=output_path,
                        processor=processor)



train_provider_T1_t_w = DataProvider(train_list_T1, [org_suffix, lab_suffix],
                                   is_pre_load=False,
                                   is_shuffle=True,
                                   # temp_dir=output_path,
                                   processor=processor_no_p,
                                is_aug=False)

train_provider_T1_w = DataProvider(train_list_T1, [org_suffix],
                                 is_pre_load=False,
                                 is_shuffle=True,
                                 # temp_dir=output_path,
                                 processor=processor_no_p,
                                   is_aug=False)

train_provider_T2_w = DataProvider(train_list_T2, [org_suffix],
                                  is_pre_load=False,
                                  is_shuffle=True,
                                  # temp_dir=output_path,
                                  processor=processor_no_p,
                                    is_aug=False)

In [14]:
gen = Generator(2, 4, 16, use_bn=use_bn)
disc = Discriminator(1, 4, 16, use_bn=use_bn)
model = GANModel([gen, disc], org_suffix, lab_suffix, g_alpha=g_alpha, g_lambda=g_lambda, g_beta=g_beta, dropout=0)
gen_lr = StepDecayLearningRate(learning_rate=learning_rate,
                           decay_step=10,
                           decay_rate=0.8,
                           data_size=train_provider_T1.size + train_provider_T2.size,
                           batch_size=batch_size)
disc_lr = StepDecayLearningRate(learning_rate=learning_rate,
                           decay_step=10,
                           decay_rate=0.8,
                           data_size=train_provider_T1.size + train_provider_T2.size,
                           batch_size=batch_size)
gen_optimizer = tf.keras.optimizers.Adam(gen_lr)
disc_optimizer = tf.keras.optimizers.Adam(disc_lr)
trainer = Trainer(model)

2023-01-29 23:53:50.655044: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/padamnoo/.conda/envs/general/lib/python3.10/site-packages/cv2/../../lib64:
2023-01-29 23:53:50.676992: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusparse.so.11'; dlerror: libcusparse.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/padamnoo/.conda/envs/general/lib/python3.10/site-packages/cv2/../../lib64:
2023-01-29 23:53:50.677141: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the re

In [15]:
# train
results = trainer.train(train_provider_T1, train_provider_T2, valid_provider_T1, valid_provider_T2, train_provider_T1_t, valid_provider_T1_t,
                        train_provider_T1_w, train_provider_T2_w,train_provider_T1_t_w,
                        epochs=epochs,
                       batch_size=batch_size,
                       mini_batch_size=mini_batch_size,
                       output_path=output_path,
                       optimizer=[gen_optimizer, disc_optimizer],
                       learning_rate=[gen_lr, disc_lr],
                       eval_frequency=eval_frequency)

Start training: epochs 1, learning rate ['0.001', '0.001'], batch size 1, mini-batch size 1, training data 2588, validation data 576.
<class 'dict'>


AttributeError: module 'numpy' has no attribute 'float'